# Explore here

In [ ]:
#!pip install scikit-learn
#!pip install matplotlib
#!pip install opencv-python
#!pip install pillow
!pip install tensorflow


In [ ]:
import os
import json
import shutil
from sklearn.model_selection import train_test_split
import zipfile
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping


1. Definir las rutas

In [ ]:
zip_path = '/workspaces/Proyecto_deep_albaebauluz/data/train.zip'   # Ruta del archivo zip
extract_path = '/workspaces/Proyecto_deep_albaebauluz/data/train'    # Carpeta donde descomprimiremos las imágenes
base_dir = '/workspaces/Proyecto_deep_albaebauluz/data/organized_data'  # Carpeta donde organizaremos el conjunto de datos


2. Descomprimir el archivo zip

In [ ]:
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
print("Descompresión completa.")

# Revisar el contenido de la carpeta después de la descompresión
extract_path = '/workspaces/Proyecto_deep_albaebauluz/data/train'  


3. Obtener la lista de imágenes

In [ ]:
all_images = os.listdir(extract_path)
print("Número de imágenes encontradas:", len(all_images))
print("Primeros 10 nombres de archivos:", all_images[:10])  # Muestra los primeros 10 nombres


4. Filtrar las listas de perros y gatos

In [ ]:
dog_images = [img for img in all_images if 'dog' in img]
cat_images = [img for img in all_images if 'cat' in img]

print("Número de imágenes de perros:", len(dog_images))
print("Número de imágenes de gatos:", len(cat_images))


5. Actualizar la ruta después de verificar la estructura

In [ ]:
extract_path = '/workspaces/Proyecto_deep_albaebauluz/data/train/train'  

6. Obtener la lista de imágenes

In [ ]:
all_images = os.listdir(extract_path)
print("Número de imágenes encontradas:", len(all_images))
print("Primeros 10 nombres de archivos:", all_images[:10])  


7. Filtrar las listas de perros y gatos

In [ ]:
dog_images = [img for img in all_images if 'dog' in img]
cat_images = [img for img in all_images if 'cat' in img]

print("Número de imágenes de perros:", len(dog_images))
print("Número de imágenes de gatos:", len(cat_images))


8. Definir la carpeta base para organizar los datos

In [ ]:
base_dir = '/workspaces/Proyecto_deep_albaebauluz/data/organized_data'  # Carpeta donde organizo el conjunto de datos

# I- Dividir en entrenamiento y validación (80% para entrenamiento y 20% para validación)

In [ ]:
train_dogs, val_dogs = train_test_split(dog_images, test_size=0.2, random_state=42)
train_cats, val_cats = train_test_split(cat_images, test_size=0.2, random_state=42)


# II- Crear carpetas de destino

In [ ]:
os.makedirs(os.path.join(base_dir, 'train/dog'), exist_ok=True)
os.makedirs(os.path.join(base_dir, 'train/cat'), exist_ok=True)
os.makedirs(os.path.join(base_dir, 'val/dog'), exist_ok=True)
os.makedirs(os.path.join(base_dir, 'val/cat'), exist_ok=True)


# III- Mover archivos a las carpetas correspondientes

In [ ]:
def move_files(file_list, source_dir, target_dir):
    for file_name in file_list:
        shutil.move(os.path.join(source_dir, file_name), os.path.join(target_dir, file_name))


# Mover imágenes de perros y gatos a las carpetas correspondientes
move_files(train_dogs, extract_path, os.path.join(base_dir, 'train/dog'))
move_files(val_dogs, extract_path, os.path.join(base_dir, 'val/dog'))
move_files(train_cats, extract_path, os.path.join(base_dir, 'train/cat'))
move_files(val_cats, extract_path, os.path.join(base_dir, 'val/cat'))

print("Organización completa. Las imágenes están listas para el entrenamiento y la validación.")



1. Visualiza la información de entrada

# Visualizar las primeras nueve imágenes de perros y gatos
def plot_images(images_folder, label, num_images=9):
    images = [img for img in os.listdir(images_folder) if label in img][:num_images]
    plt.figure(figsize=(10, 10))
    for i, img_name in enumerate(images):
        img_path = os.path.join(images_folder, img_name)
        img = Image.open(img_path)  # Cargar imagen con Pillow
        plt.subplot(3, 3, i+1)
        plt.imshow(img)
        plt.axis('off')
    plt.show()


In [ ]:
# Rutas para las imágenes de perros y gatos
dog_images_folder = '/workspaces/Proyecto_deep_albaebauluz/data/train/train'  
cat_images_folder = '/workspaces/Proyecto_deep_albaebauluz/data/train/train'  

print("Primeras nueve imágenes de perros:")
plot_images(dog_images_folder, 'dog')

print("Primeras nueve imágenes de gatos:")
plot_images(cat_images_folder, 'cat')


2. Configurar los generadores de datos para entrenamiento y validación

In [ ]:
train_datagen = ImageDataGenerator(rescale=1.0/255.0)
val_datagen = ImageDataGenerator(rescale=1.0/255.0)


3. Crear los flujos de datos para entrenamiento y validación desde las carpetas

In [ ]:
train_dir = '/workspaces/Proyecto_deep_albaebauluz/data/organized_data/train'
val_dir = '/workspaces/Proyecto_deep_albaebauluz/data/organized_data/val'

trdata = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)

tsdata = val_datagen.flow_from_directory(
    val_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)




# Obtener un lote de datos de entrenamiento
images, labels = next(trdata)

# Mostrar las primeras 9 imágenes
plt.figure(figsize=(10, 10))
for i in range(9):
    plt.subplot(3, 3, i+1)
    plt.imshow(images[i])
    plt.title("Dog" if labels[i] == 1 else "Cat")
    plt.axis('off')
plt.show()


# IV. Modelo simplificado

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(128,128,3)))
model.add(MaxPool2D((2,2)))
model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(MaxPool2D((2,2)))
model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(MaxPool2D((2,2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=Adam(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])
model.summary()


In [ ]:
# Resumen del modelo para confirmar la arquitectura
model.summary()


In [ ]:
# Definir early stopping para evitar sobreajuste
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


In [ ]:
# Entrenamiento del modelo
history = model.fit(
    trdata,
    validation_data=tsdata,
    epochs=20,
    callbacks=[early_stopping]
)


In [ ]:
# Evaluación del modelo en los datos de validación
loss, accuracy = model.evaluate(tsdata)
print(f"Precisión en el conjunto de validación: {accuracy:.2f}")



# V. Optimización del modelo

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import load_model



In [ ]:
# Crear los callbacks
checkpoint = ModelCheckpoint(
    "best_model.keras",         # Cambio la extensión a .keras
    monitor="val_accuracy",     # Métrica para monitorear
    save_best_only=True,        # Guardo el mejor modelo
    mode="max",                 # Guardo el modelo con la precisión de validación más alta
    verbose=1
)

early_stopping = EarlyStopping(
    monitor="val_loss",         # Monitorea la pérdida en el conjunto de validación
    patience=5,                 # Detiene el entrenamiento si no hay mejora en 5 épocas
    restore_best_weights=True   # Restaura el mejor peso al final
)


In [ ]:
# Entrenar el modelo con los callbacks
history = model.fit(
    trdata,
    validation_data=tsdata,
    epochs=20,
    callbacks=[checkpoint, early_stopping]
)


In [ ]:
# Cargar el mejor modelo guardado
best_model = load_model("best_model.keras")


In [ ]:
# Evaluar el modelo en el conjunto de prueba
test_loss, test_accuracy = best_model.evaluate(tsdata)
print(f"Precisión en el conjunto de prueba: {test_accuracy:.2f}")


In [ ]:
# Realizar predicciones en el conjunto de prueba
predicciones = best_model.predict(tsdata)
predicciones_clases = (predicciones > 0.5).astype("int32")  # Umbral para convertir probabilidades en clases


In [ ]:
# Mostrar los resultados
print("Predicciones de las primeras 10 imágenes de prueba:", predicciones_clases[:10])



# VI. Guardar el modelo

In [ ]:
# Crear la carpeta 'models' 
os.makedirs('models', exist_ok=True)

# Guardo el modelo en la carpeta 'models'
model_path = '/workspaces/Proyecto_deep_albaebauluz/models/best_model.keras'
best_model.save(model_path)

print(f"Modelo guardado en: {model_path}")



NOTA: Tras realizar todo el ejercicio, no me deja guardar los resultados porque pesa mucho los datos. He intentado solucionar el problema, pero al no disponer de la plataforma local me ha sido imposible solventarlo.

Enumerating objects: 25037, done.
Counting objects: 100% (25037/25037), done.
Delta compression using up to 2 threads
Compressing objects: 100% (25029/25029), done.
Writing objects: 100% (25032/25032), 1.22 GiB | 36.53 MiB/s, done.
Total 25032 (delta 16), reused 25006 (delta 1), pack-reused 0
remote: Resolving deltas: 100% (16/16), completed with 1 local object.
remote: error: Trace: bf389b2be3ac8a7631ac3d7f862016c9f1c7dd22a22c903a5ba074ae0bcb00cc
remote: error: See https://gh.io/lfs for more information.
remote: error: File src/best_model.keras is 193.12 MB; this exceeds GitHub's file size limit of 100.00 MB
remote: error: File data/train.zip is 543.16 MB; this exceeds GitHub's file size limit of 100.00 MB
remote: error: File models/best_model.keras is 193.12 MB; this exceeds GitHub's file size limit of 100.00 MB
remote: error: File src/models/best_model.keras is 193.12 MB; this exceeds GitHub's file size limit of 100.00 MB
remote: error: GH001: Large files detected. You may want to try Git Large File Storage - https://git-lfs.github.com.
To https://github.com/4GeeksAcademy/Proyecto_deep_albaebauluz
 ! [remote rejected]   main -> main (pre-receive hook declined)
error: failed to push some refs to 'https://github.com/4GeeksAcademy/Proyecto_deep_albaebauluz'
